In [ ]:
# normalize (not gonna use)
def normalizeLineProfs(dataname,linepath,imagepath,celldia=200):
    imgpthname = '%s%s/' %(imagepath,dataname)
    linepthname = '%s%s/' %(linepath,dataname)
    
    #load features and profiles, and add kkdists (in pixels)
    df_feat = pd.read_csv(linepthname+dataname+'_feat.csv')
    df_feat = df_feat.drop(df_feat.columns[0],axis=1)
    
    df_prof = pd.read_csv(linepthname+dataname+'_prof.csv')
    df_prof = df_prof.drop(df_prof.columns[0],axis=1)
    df_prof[['prof488','prof560']] = df_prof[['prof488','prof560']].applymap(lambda x:np.array(eval(re.sub(r'(\d)\s+(\d)', r'\1,\2',x))))
    
    kkdists = []
    for i in df_prof.index:
        ids = df_prof.iloc[i][['kinid1','kinid2']].values
        fr = df_prof.iloc[i]['fr']
        feat_sel = df_feat.loc[(df_feat['frame']==fr) & ((df_feat['kinid']==ids[0]) | (df_feat['kinid']==ids[1]))]
        kkdists.append(np.linalg.norm(np.diff(feat_sel[['x','y']].values,axis=0)))
    
    df_prof['kkdist'] = kkdists
    
    
    # Determine center of the cell
    
    
    
    #load first timepoint images to compute cytoplasmic background level
    imgnamelist = np.array(listdir(imgpthname))
    
    mask488 = [len(re.findall('\_t001_z\d*_c001',imgname))>0 for imgname in imgnamelist]
    mask560 = [len(re.findall('\_t001_z\d*_c002',imgname))>0 for imgname in imgnamelist]
    
    imgnamelist0_488 = imgnamelist[np.array(mask488)]
    imgnamelist0_560 = imgnamelist[np.array(mask560)]
    
    imgs488 = []
    filtimgs488 = []
    imgs560 = []
    filtimgs560 = []
    
    for fname in imgnamelist0_488:
        temp = plt.imread(imgpthname+fname)
        imgs488.append(temp)
    
    for img488 in imgs488:
        filtimgs488.append(gaussian_filter(np.double(img488),3))

    gm = gaussian_mixture.GaussianMixture(n_components=2)
    temp = np.array(filtimgs488)
    gm.fit(np.array([temp.flatten()]).transpose())
    cytobg488 = np.max(gm.means_)
    
    
    for fname in imgnamelist0_560:
        temp = plt.imread(imgpthname+fname)
        imgs560.append(temp)
    
    for img560 in imgs560:
        filtimgs560.append(gaussian_filter(np.double(img560),3))

    gm = gaussian_mixture.GaussianMixture(n_components=2)
    temp = np.array(filtimgs560)
    gm.fit(np.array([temp.flatten()]).transpose())
    cytobg560 = np.max(gm.means_)
    